In [1]:
import h5py
import numpy as np
np.random.seed(1337)
from keras.optimizers import SGD

Using Theano backend.


In [2]:
h5f = h5py.File('preprocessed_data_winSize100_winShift10.h5','r')
training_data = h5f['training_data'][:]
training_output = h5f['training_output'][:]
testing_data = h5f['testing_data'][:]
testing_output = h5f['testing_output'][:]

h5f = h5py.File('preprocessed_data_winSize100_winShift10_mean_sd.h5','r')
mean = h5f['mean'][:]
sd = h5f['sd'][:]
h5f.close()

print(training_data.shape,training_output.shape,testing_data.shape,testing_output.shape)

print("Concatenating training")
training = np.concatenate([training_data, training_output.reshape(training_output.shape[0],1)], axis=1)
print("Deleting training")
del training_data
del training_output
print("Shuffling training")
np.random.shuffle(training)

print("Concatenating testing")
testing = np.concatenate([testing_data, testing_output.reshape(testing_output.shape[0],1)], axis=1)
print("Deleting testing")
del testing_data
del testing_output
print("Shuffling Testing")
np.random.shuffle(testing)

print("Getting back train data and output")
training_data=training[:,:-1]
training_output=training[:,-1]
print("Deleting training")
del training

print("Getting back test data and output")
testing_data=testing[:,:-1]
testing_output=testing[:,-1]
print("Deleting testing")
del testing

print(training_data.shape,training_output.shape,testing_data.shape,testing_output.shape)

(218540, 1200) (218540,) (93660, 1200) (93660,)
Concatenating training
Deleting training
Shuffling training
Concatenating testing
Deleting testing
Shuffling Testing
Getting back train data and output
Deleting training
Getting back test data and output
Deleting testing
(218540, 1200) (218540,) (93660, 1200) (93660,)


In [8]:
testing_data=testing_data-mean
training_data=training_data-sd

In [17]:

batch_size = 512
nb_classes = 7
nb_epoch = 5
learning_rate = 0.0001

sgd = SGD(lr=lr)

training_data = training_data.astype('float32')
testing_data = testing_data.astype('float32')

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce 840M (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python3.5/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [11]:
Y_train = np_utils.to_categorical(training_output, nb_classes)
Y_test = np_utils.to_categorical(testing_output, nb_classes)
print(Y_test.shape)

/usr/local/lib/python3.5/dist-packages/keras/utils/np_utils.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


(93660, 7)


In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(training_data.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_7 (Dense)                  (None, 512)           614912      dense_input_3[0][0]              
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 512)           0           dense_7[0][0]                    
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 512)           0           activation_7[0][0]               
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 7)             3591        dropout_5[0][0]                  
___________________________________________________________________________________________

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(training_data, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(testing_data, Y_test))
score = model.evaluate(testing_data, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 218540 samples, validate on 93660 samples
Epoch 1/5
218540/218540 [==============================] - 8s - loss: 2.3843 - acc: 0.1423 - val_loss: 1.9593 - val_acc: 0.1431
Epoch 2/5
218540/218540 [==============================] - 8s - loss: 1.9526 - acc: 0.1422 - val_loss: 1.9512 - val_acc: 0.1427
Epoch 3/5
218540/218540 [==============================] - 8s - loss: 1.9472 - acc: 0.1432 - val_loss: 1.9468 - val_acc: 0.1427
Epoch 4/5
218540/218540 [==============================] - 8s - loss: 1.9461 - acc: 0.1423 - val_loss: 1.9473 - val_acc: 0.1427
Epoch 5/5
218540/218540 [==============================] - 8s - loss: 1.9459 - acc: 0.1413 - val_loss: 1.9473 - val_acc: 0.1427
Test score: 1.94725065188
Test accuracy: 0.14269698911
